In [46]:
import qsharp
import qsharp_widgets
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-multipliers")
qsharp.init(project_root="../lib/")

@cache.memoize()
def estimate_resources_divide_pareto(op, n):
    if ";" in op:
        divider_type, adder = op.split(";")
        if divider_type == "Restoring":
            est = qsharp.estimate(f"QuantumArithmetic.TMVH2019Test.RunForRE_Restoring({n},{adder})", params={"estimateType": "frontier"} )
        elif divider_type == "NonRestoring":
            est = qsharp.estimate(f"QuantumArithmetic.TMVH2019Test.RunForRE_NonRestoring({n},{adder})", params={"estimateType": "frontier"})
        else:
            raise ValueError("Unknown divider_type")
    elif op == "AKBF2011.Divide_Restoring":
        est = qsharp.estimate(f"QuantumArithmetic.AKBF2011Test.RunForRE_Divide_Restoring({n})", params={"estimateType" : "frontier"})
    else:
        raise ValueError("Unknown op")
    return est

ops_and_max_n = [
  ("NonRestoring;Std.Arithmetic.RippleCarryCGIncByLE", "NonRestoring+Gidney", 2**14),
  ("NonRestoring;Std.Arithmetic.RippleCarryTTKIncByLE", "NonRestoring+TTK", 2**14),
  ("NonRestoring;QuantumArithmetic.CDKM2004.Add", "NonRestoring+CDKM", 2**12),
  ("NonRestoring;QuantumArithmetic.JHHA2016.Add_Mod2N", "NonRestoring+JHHA", 2**12),
  ("Restoring;Std.Arithmetic.RippleCarryCGIncByLE", "Restoring+Gidney", 2**12),
  ("Restoring;Std.Arithmetic.RippleCarryTTKIncByLE", "Restoring+TTK", 2**14),
  ("Restoring;QuantumArithmetic.CDKM2004.Add", "Restoring+CDKM", 2**12),
  ("Restoring;QuantumArithmetic.JHHA2016.Add_Mod2N", "Restoring+JHHA", 2**12),
  ("AKBF2011.Divide_Restoring", "AKBF", 2**12),
]

In [50]:
n = 2**9
results = {} 
i = 0 
for op, _, _ in ops_and_max_n:
    results[i] = estimate_resources_divide_pareto(op, n)
    i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, _ in ops_and_max_n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…